Import Libraries
TO DO
1) Describe Grid Location bottom left hand location and Cell Size
2) Plot the vector using x y coordinates.
3) Compare to Raster Stat Results, need to make sure that it only includes the cells fully inside (not on the border of the vector.

In [18]:
import numpy as np
print (np.__version__)
from osgeo import gdal
from scipy import stats
import timeit
from rasterstats import zonal_stats
from shapely.geometry import Point, Polygon
from osgeo import ogr
from math import ceil,floor

1.11.3


<img src="data/thumb.png" alt="Drawing" style="width: 300px;"/>

Load test data

In [4]:
r = gdal.Open("data\\nlcd_2001_landcover_2011_edition_2014_10_sub.tif")
shape = "data\\test.shp"
g = np.array(r.GetRasterBand(1).ReadAsArray())
csze = r.GetGeoTransform()[1]
ulX = r.GetGeoTransform(0)
ulY = r.GetGeoTransform(3)
print (uly)

(1599633.055684202, 30.0, 0.0, 2083307.9820680085, 0.0, -30.0)


In [5]:
## I checked the top left cell in q gis and it is value = 95 bottom left = 41. I want g[0,0] = 41 so that the bottom left is 0,0
##print(g[0,0])
##g = np.flipud(g)
##print(g[0,0])

In [6]:
##stats.describe(g) 
print( (r.RasterYSize,r.RasterXSize))
print( g.shape )
##out.shape

(1734, 2501)
(1734, 2501)


Function to create data structure function takes in a grid with mxn and returns a grid of the same size summed by columns.  A vectorized version would be faster

In [7]:
def dtFZS(g,out):
    for xw in range(0,g.shape[1]):
        for yw in range(1,g.shape[0]):
            out[yw,xw] = out[yw-1,xw] + g[yw,xw]
    print("done")

In [8]:
##out = dtFZS(g)
DS_FstZS = np.zeros(g.shape,dtype='uint64')
dtFZS(g,DS_FstZS)


done


In [9]:
##stats.describe(DS_FstZS)
##out.shape

Fast Summary Statistics Function
    

In [29]:
## Create a function that takes in x1, x2, ulX, and cllsze in world coords and returns list of x locations in grid coordinates
def x_world_to_grid(x1,x2,ulX,clSze):
    x1a = min(x1,x2)
    x2a = max(x1,x2)
    
    xstart = ceil  ((x1a - ulX - (clSze/2)) / clSze)
    xstop =  floor ((x2a - ulX - (clSze/2)) / clSze)
    return(list(range(xstart,xstop)))

x_world_to_grid(510,560,500,10)



[1, 2, 3, 4]

In [32]:
def x_grid_to_world(x,ulX,clSze):
    xout = [i * clSze + ulX + (clSze/2) for i in x]
    
x_grid_to_world(x_world_to_grid(510,560,500,10),500,10)    
    

In [ ]:
## Create a function that takes in world coordinates x1, x2, y1, y2 and xpred and returns ypred linear function

In [ ]:
## Create a function that takes in y1, y2, ulY, world coord and returns y grid coord one for every x

def ygrid(x1,x2,ulX,y1,y2,ulY,clSze):
    
    
    
    

In [10]:
## Create Tuple function
def xyrange(ystart,ystop,xstart,xstop,step=1):
    if(xstop < xstart):
        step = - 1
    xtuple = list(range(xstart,xstop,step))
    ytuple = [round(( x/len(xtuple) * (ystop-ystart) ) + ystart) for x in range(0,len(xtuple))]  
    return [ytuple,xtuple]

idx = xyrange(0,100,10,5)
print(idx)
#idx = [(0,0,1,1),(0,1,0,2)]

DS_FstZS[idx]
 

[[0, 20, 40, 60, 80], [10, 9, 8, 7, 6]]


array([   0, 1194, 2068, 2484, 2851], dtype=uint64)

In [11]:
##  To DO
## This function needs to be rewritten to make v a lattice on the grid (not matching the grid locations but between the grid valeus)
## and to take coordinates

def FstZS(v,DS_FstZS,cellsize,ulx,uly):
    total = 0;    
    for p in range(1,(len(v)+0)):
        ## If moving left
        if v[p][1] > v[p-1][1]:            
            total += np.sum(DS_FstZS[xyrange(v[p][0],v[p-1][0],v[p][1],v[p-1][1])])
        ## If moving Right
        elif v[p][1] < v[p-1][1]: 
            total -= np.sum(DS_FstZS[xyrange(v[p][0],v[p-1][0],v[p][1],v[p-1][1])])     
    return int(total)

In [38]:
v = [(1608490,2038038),(1608450,2038038),(1608450,2038138),(1608490,2038138),(1608490,2038038)]

In [39]:
#g.shape
FstZS(v,DS_FstZS)
##xyrange

IndexError: index 1608450 is out of bounds for axis 0 with size 1734

In [40]:
### Import Test vector
polygon = Polygon(v) 
zonal_stats(polygon,"data\\nlcd_2001_landcover_2011_edition_2014_10_sub.tif",stats= "sum")

C:\Users\smh362\AppData\Local\Continuum\Anaconda3\lib\site-packages\rasterstats\main.py:145: FutureWarning: The value of this property will change in version 1.0. Please see https://github.com/mapbox/rasterio/issues/86 for details.
  with Raster(raster, affine, nodata, band) as rast:
C:\Users\smh362\AppData\Local\Continuum\Anaconda3\lib\site-packages\rasterstats\io.py:242: FutureWarning: GDAL-style transforms are deprecated and will not be supported in Rasterio 1.0.
  self.affine = guard_transform(self.src.transform)


[{'sum': 246.0}]

Pretty Close Not sure why these are slightly different end